In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from nd2reader import ND2Reader
from tifffile import imread
from matplotlib import pyplot as plt
from skimage.measure import regionprops
from scipy.ndimage import gaussian_filter

from glcm_utils import get_glcm_feature_names, get_glcm_features_masked, get_glcm_features_per_label
from glcm_utils import DEFAULT_PROPERTIES, _get_uint_dtype_for_masked_img

# Get GLCM features for IMR90 overview or spinning disk data

## 1) load images and masks

In [ ]:
## Spinning disk (only ctrl)
# base_path = Path('/data/cooperation_data/ArgyrisPapantonis-nuclear_architecture/Hartmann_Harz/IMR90_30112022/')
# seg_path = base_path / 'segmentation_cellpose_d120_ft06'
# img_files = sorted(base_path.glob('*.nd2'))

## Stitched overviews
base_path = Path('/scratch/hoerl/20230507_imr90_stitching/20230507_imr90_ov_stitch_output/')
seg_path = base_path / 'segmentation_cellpose_maxproj_d70_ft07'
img_files = sorted(base_path.glob('*.tif'))

# img_files

In [ ]:
imgs, masks = [], []
for img_file, mask_file in zip(img_files, sorted(seg_path.glob('*.tif'))):

    if img_file.suffix == '.nd2':
        with ND2Reader(str(img_file)) as reader:
            img = np.array(reader[0])
    elif img_file.suffix == '.tif':
        img = imread(img_file).max(axis=0)

    imgs.append(img)
    masks.append(imread(mask_file))
    print(f'loaded {img_file}')

## 2) Normalize images

In [ ]:
def normalize(arr, min_, max_):
    '''
    non-clipping linear normalization, min_ will be set to 0, max_ to 1 in the output 
    '''
    return (arr - min_) / (max_ - min_)

# intensity normalization per object 
def normalize_per_object(img, mask):
    img = img.copy().astype(np.float32)
    for rprop in regionprops(mask, img):
        img[rprop.slice][rprop.image] -= rprop.min_intensity
        img[rprop.slice][rprop.image] /= (rprop.max_intensity - rprop.min_intensity)
    return img

In [ ]:
percentiles=(2.5, 99.8)

imgs_normalized = []
for img in imgs:
    mi, ma = np.percentile(img[img>0], percentiles)
    # normalize imgs
    img = gaussian_filter(img, 0.5)
    imgs_normalized.append(normalize(img, mi, ma))

imgs_normalized_perobject = []
for img, mask in zip(imgs, masks):
    imgs_normalized_perobject.append(normalize_per_object(img, mask))

## 3) GLCM features

In [ ]:
distances = [2, 4, 8, 16]
angles = [0, np.pi/2]

df_texture = pd.DataFrame()

for idx in range(len(imgs)):

    img_for_glcm = imgs_normalized[idx]

    ## Alternative: use per-object normalized -> 0.5px blur has not been applied yet
    # img_for_glcm = imgs_normalized_perobject[idx]
    # img_for_glcm = gaussian_filter(img_for_glcm, 0.5)

    dfi = pd.DataFrame.from_dict(get_glcm_features_per_label(img_for_glcm, masks[idx], distances, angles))
    dfi['file'] = img_files[idx].name

    if len(dfi) > 0:
        df_texture = df_texture.append(dfi.set_index(['file', 'label']))

df_texture.columns = ["tex_" + c for c in df_texture.columns]

In [ ]:
df_texture

In [ ]:
df_texture.to_csv(seg_path / 'texture_feats.csv')
# df_texture.to_csv(seg_path / 'texture_feats_normalized_per_cell.csv')

## 4) Shape / intensity features

In [ ]:
from skimage.measure import regionprops_table

props_to_extract = ('label', 'area', 'eccentricity', 'mean_intensity', 'bbox')

df_other = pd.DataFrame()

for idx in range(len(imgs)):
    dfi = pd.DataFrame.from_dict(regionprops_table(masks[idx], imgs_normalized[idx], props_to_extract))
    dfi['file'] = img_files[idx].name

    if len(dfi) > 0:
        df_other = df_other.append(dfi.set_index(['file', 'label']))
    
df_other.columns = ["other_" + c for c in df_other.columns]

In [ ]:
df_other

In [ ]:
df_other.to_csv(seg_path / 'other_feats.csv')

# Testing code

In [ ]:
# test dtype for GLCM
_get_uint_dtype_for_masked_img(2**32-1), _get_uint_dtype_for_masked_img(2**32)

In [ ]:
# GLCM feature options
props = DEFAULT_PROPERTIES

distances = [2, 4, 7, 12, 16]
angles = [0, np.pi/2]

get_glcm_feature_names(distances, angles, props)

### Simulate simple objects with varying texure

In [ ]:
shape = (512, 512)

radii = [30, 40, 50]
coords = [
    [124, 421],
    [421, 421],
    [256, 210],
]

lab = np.zeros(shape, dtype=int)
img = np.zeros(shape, dtype=np.float32)

# select circlular area of each object
for i, (r, x) in enumerate(zip(radii, coords), 1):
    sel = np.linalg.norm(np.stack(np.meshgrid(*(np.arange(s) for s in lab.shape)), -1) - np.array(x), axis=-1) < r
    lab[sel] = i


# range for uniform random values within each object
ranges = [
    [0.4, 0.5],
    [0.4, 0.5],
    [0.1, 0.8],
]
    
for r, (mi, ma) in zip(regionprops(lab, img), ranges):
    img[r.slice][r.image] = np.random.uniform(mi, ma, np.sum(r.image))

fig, axs = plt.subplots(ncols=2)
axs[0].imshow(lab)
axs[1].imshow(img)

In [ ]:
# test intensity normalization per object

img_norm = normalize_per_object(img, lab)
fig, axs = plt.subplots(ncols=2)
axs[0].imshow(img)
axs[1].imshow(img_norm)

### Test GLCM calculation per object

In [ ]:
res = get_glcm_features_per_label(img, lab, distances=[2], n_bins=64)
pd.DataFrame.from_dict(res).set_index('label')

In [ ]:
# check for error on empty GLCM
# distance > image size

res = get_glcm_features_per_label(img, lab, distances=[513], n_bins=64)
pd.DataFrame.from_dict(res).set_index('label')